## Scenario 2 -  Multi-tenancy

### **SupportWizard** - Support Analyis SaaS Platform

- Allow its users to sign up and upload their own customer support data
- The platform can then analyse the information to identify where they could improve their support schema, and even their own products 

### Solution 

Each end user will have their own isolated "space", to which they can uplaod data. Then, they can use SupportWizard dashboards / platform to see analyses of their own data. 


### Create the collection


In [1]:
import os
import weaviate
from weaviate.classes.config import Property, DataType, Configure

client = weaviate.connect_to_local(
    headers={"X-Cohere-Api-Key": os.getenv("COHERE_API_KEY")}
)

collection_name = "SupportChatMT"

# For re-running the demo only: Delete existing collection if it exists
client.collections.delete(collection_name)

# Create a new collection with specified properties and vectorizer configuration
chunks = client.collections.create(
    name=collection_name,
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="dialogue_id", data_type=DataType.INT),
        Property(name="company_author", data_type=DataType.TEXT),
        Property(name="created_at", data_type=DataType.DATE),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="text_with_metadata",
            source_properties=["text", "company_author"],
            vector_index_config=Configure.VectorIndex.hnsw(),
            api_endpoint="http://host.docker.internal:11434",
            model="nomic-embed-text",
        )
    ],
    generative_config=Configure.Generative.cohere(model="command-r-plus"),
    multi_tenancy_config=Configure.multi_tenancy(enabled=True, auto_tenant_creation=True)
)

In [2]:
import h5py
import json
import numpy as np


def get_hdf5_obj(file_path):
    with h5py.File(file_path, "r") as hf:
        for uuid in hf.keys():
            src_obj = hf[uuid]

            # Get the object properties
            properties = json.loads(src_obj["object"][()])

            # Get the vector(s)
            vectors = {}
            for key in src_obj.keys():
                if key.startswith("vector_"):
                    vector_name = key.split("_", 1)[1]
                    vectors[vector_name] = np.asarray(src_obj[key])

            yield uuid, properties, vectors

In [3]:
from tqdm import tqdm

tenant_names = ["AcmeCo", "Globex", "Initech", "UmbrellaCorp", "WayneEnterprises"]

with client.batch.fixed_size(batch_size=200) as batch:
    for uuid, properties, vectors in tqdm(get_hdf5_obj("data/twitter_customer_support_nomic.h5")):
        # Assign a tenant to object based on the company author
        tenant_index = len(properties['company_author']) % 5
        tenant_name = tenant_names[tenant_index]

        # Add the object to the batch
        batch.add_object(
            collection=collection_name,
            uuid=uuid,
            properties=properties,
            vector={"text_with_metadata": vectors["text_with_metadata"]},
            tenant=tenant_name  # <===== This is the only line that changes
        )


50000it [00:17, 2833.18it/s]


In [4]:
print(f"Processed {len(client.batch.results.objs.all_responses)} objects.")

Processed 50000 objects.


/Users/jphwang/code/demo-code/202501-vectordb-patterns/.venv/lib/python3.10/site-packages/weaviate/warnings.py:213: DeprecationWarning: Dep020: The `all_responses` attribute in the `BatchResults` object is deprecated and will be removed by Q4 2024. Please instead use the `errors` and `uuids` attributes.
  warnings.warn(


In [5]:
if len(client.batch.failed_objects) > 0:
    print("*" * 80)
    print(f"***** Failed to add {len(client.batch.failed_objects)} objects *****")
    print("*" * 80)
    print(client.batch.failed_objects[:3])

In [6]:
support_chats = client.collections.get(collection_name)

In [7]:
support_chats.tenants.get()


{'WayneEnterprises': Tenant(name='WayneEnterprises', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'Globex': Tenant(name='Globex', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'AcmeCo': Tenant(name='AcmeCo', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'Initech': Tenant(name='Initech', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'UmbrellaCorp': Tenant(name='UmbrellaCorp', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>)}

In [11]:
tenant_data = support_chats.with_tenant(tenant_names[0])

In [12]:
response = tenant_data.query.fetch_objects(limit=2, include_vector=True)

In [13]:
print(response.objects[0].uuid)

00023216-25c8-5d2c-bd5e-fa5bd486b566


In [14]:
for k, v in response.objects[0].properties.items():
    print(f"\n|| {k} || \n{v}")


|| text || 
User_124655: Argh, Hey Amazon! WHERE IS MY PACKAGE? Arriving between Nov 1&amp;3 doesn't mean shop Nov. 2 and maybe you'll get it the next day. WTF?
AmazonHelp: If your order doesn't arrive on the 3rd by 8:00, please contact us for options: https://t.co/EKXRLsnxJu ^AF

|| company_author || 
AmazonHelp

|| created_at || 
2017-11-01 14:53:08+00:00

|| dialogue_id || 
39592


In [15]:
for k, v in response.objects[0].vector.items():
    print(k, v[:3])

text_with_metadata [-0.5452395677566528, 1.116987705230713, -2.7637276649475098]


### Queries

In [16]:
def display_objects(response):
    for o in response.objects:
        print(o.uuid, "\n")
        print(o.properties["text"][:200], "\n")

In [17]:
response = tenant_data.query.near_text("return process", limit=3)
display_objects(response)

3e350821-8e26-5746-8769-408223cef412 

User_187419: I've returned one product through self ship for which i beared courier charges. What is the procedure to take refund of the same?
AmazonHelp: Kindly reach out to our support team here: ht 

0de3be60-85ab-5011-a278-a5aa857992f5 

User_178546: Hi, how long do returns usually take to get back to you? Tracking status hasn't changed for 2 days on that pass my parcel site
AmazonHelp: After the carrier receives the item, it can take 

3247d280-de61-515f-a388-caaac81770c8 

User_206228: please check the DM sent to @AmazonHelp abd revert.
AmazonHelp: Hi, we have responded to you via DM. Please refer. ^RD
User_206228: @AmazonHelp Your response is of no help to me.I have al 



In [18]:
response = tenant_data.query.bm25("return process", limit=3)
display_objects(response)

31a21f1b-61aa-5c01-bcbf-6dce68b2cb5e 

User_119904: @115850 I have bought a product and now it's size is not matching I want to return it and also requested return process.
AmazonHelp: You may refer here: https://t.co/M27c4qF86m for detail 

09f5607c-2d4e-54f5-bcbc-74012994eb80 

User_295654: I have purchased Lenovo k8 note ....  But heating mobile I want to return it what is process
AmazonHelp: We're responding to your query via DM. Would request you to have a look at it. ^AB 

3c0cba72-874e-5119-aa3d-42dc3b480127 

User_233994: Currently trying to return a £7.99 book to you and there are no options to do so that don't cost me £3.99 which is absolutely appalling? AS IF it costs £4 to send something back?!?!
Amazo 



In [19]:
response = tenant_data.query.hybrid("return process", limit=3)
display_objects(response)

31a21f1b-61aa-5c01-bcbf-6dce68b2cb5e 

User_119904: @115850 I have bought a product and now it's size is not matching I want to return it and also requested return process.
AmazonHelp: You may refer here: https://t.co/M27c4qF86m for detail 

0de3be60-85ab-5011-a278-a5aa857992f5 

User_178546: Hi, how long do returns usually take to get back to you? Tracking status hasn't changed for 2 days on that pass my parcel site
AmazonHelp: After the carrier receives the item, it can take 

09f5607c-2d4e-54f5-bcbc-74012994eb80 

User_295654: I have purchased Lenovo k8 note ....  But heating mobile I want to return it what is process
AmazonHelp: We're responding to your query via DM. Would request you to have a look at it. ^AB 



In [20]:
response = tenant_data.generate.fetch_objects(
    limit=20,
    grouped_task="What patterns are we seeing here in these issues?"
)

In [21]:
print(response.generated)

Based on the provided data, there are several patterns that can be observed in the issues presented:

- **Late or missing deliveries:** Customers are inquiring about the status of their packages, with some expressing frustration over late or missing deliveries.
- **Customer support:** Many users are reaching out to the companies' customer support teams for assistance with various issues, including refunds, product availability, and technical problems.
- **Product or service complaints:** Some customers are expressing dissatisfaction with the quality of products or services received, such as meals on board a flight or the placement of products in a store.
- **Account and payment inquiries:** Users are seeking information or assistance with account-related matters, such as tax benefits for business accounts or payment options (e.g., card installments).
- **Feedback and suggestions:** Customers are providing feedback and suggestions to the companies, such as requesting improvements to onl

In [22]:
from weaviate.classes.query import Filter

response = tenant_data.generate.fetch_objects(
    limit=20,
    grouped_task="What patterns are we seeing here in these issues?",
    filters=Filter.by_property("text").like("*new york*")
)

In [23]:
for o in response.objects:
    print(o.uuid)
    print(o.properties["text"], "\n")

048bd9e3-480c-51dd-abf9-693aad7bf78f
User_274016: Hello Saino, I was wondering where I could find your new Vegan Marshamallow and Cream for my hot chocolate? I live near York and can’t find it at Monks Cross. #HelpMe
sainsburys: ...by following this link: https://t.co/9OrDZmHtAB. Steven 2/2
sainsburys: Hi, I can confirm the mallows will be restocked in our Monks Cross store on the 24th November. Unfortunately the cream isn't available for sale in any store close by. Not to worry though you can log a product request with our buyers..1/2
User_274016: Steven 🙌🏻 you sweetie! Thanks for your help! I shall keep my eyes peeled 👀 ta v.much!
sainsburys: We're here to help, hope you have a grand day! Maclaine 

13030d41-cd5a-526b-b5b4-d25ac5247231
User_203779: what happens if you have a basic economy ticket and flight is delayed or cancelled?
Delta: We would get you on the next available flight. A Basic Economy ticket would only inhibit voluntary changes. *CBK
User_203779: So in other words you 

In [24]:
print(response.generated)

Based on the provided data, there are a few patterns that can be observed:

- Many of the issues are related to travel, particularly flights and accommodations. This includes flight delays, cancellations, overbooking, seating issues, lost luggage, and issues with hotel bookings.
- There are also some issues related to product availability and restocking, such as the Sainsburys and Tesco examples.
- In terms of customer service, there is a mix of both positive and negative feedback. Some customers express appreciation for helpful responses, while others criticize rude staff or unsatisfactory resolutions.
- A few of the issues involve multiple parties, such as the British Airways example where the customer had to deal with both the airline and the hotel for their issue.
- Social media platforms, particularly Twitter, seem to be a common channel for customers to reach out to companies and seek assistance or share feedback.

Overall, these patterns suggest that travel-related issues, produ

### Tenant management




### Demo application

- Outside of the notebook
